<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/181-Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieval

Retrieval is the centerpiece of our retrieval augmented generation (RAG) flow.

Let's get our vectorDB from before.

In [ ]:
!pip install openai==0.28
!pip install python-dotenv
!pip install langchain langchain-community

In [2]:
!echo "OPENAI_API_KEY=sk-" >> .env
!source /content/.env

In [3]:
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

### OpenAI API Key 설정 및 Key 값 출력

In [ ]:
print("OpenAI API Key 출력:", openai.api_key) #OpenAI API Key 설정 및 Key 값 출력

### OpenAI 설치  Ver 확인

In [ ]:
!pip show openai|grep Version #OpenAI 설치  Ver 확인

## Vectorstore retrieval


In [5]:
# import os
# import openai
# import sys

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
# !pip install lark

In [ ]:
!mkdir docs/chroma

### Similarity Search

In [ ]:
!pip install chromadb

In [ ]:
!pip install tiktoken

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'

In [12]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
print(vectordb._collection.count())

0


In [14]:
texts = [
    """광대버섯(Amanita phalloides)은 크고 인상적인 후성(위) 자실체(담자과체)를 가지고 있습니다.""",
    """큰 자실체를 가진 버섯은 Amanita phalloides입니다. 일부 품종은 모두 흰색입니다.""",
    """A. phalloides, 일명 Death Cap은 알려진 모든 버섯 중에서 가장 독성이 강한 버섯 중 하나입니다.""",
]

In [15]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [16]:
question = "큰 자실체를 가진 순백색 버섯에 대해 알려주세요"

In [17]:
smalldb.similarity_search(question, k=2)

[Document(page_content='큰 자실체를 가진 버섯은 Amanita phalloides입니다. 일부 품종은 모두 흰색입니다.'),
 Document(page_content='A. phalloides, 일명 Death Cap은 알려진 모든 버섯 중에서 가장 독성이 강한 버섯 중 하나입니다.')]

In [18]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(page_content='큰 자실체를 가진 버섯은 Amanita phalloides입니다. 일부 품종은 모두 흰색입니다.'),
 Document(page_content='A. phalloides, 일명 Death Cap은 알려진 모든 버섯 중에서 가장 독성이 강한 버섯 중 하나입니다.')]

## 다른 유형의 검색

VectorDB가 문서를 검색하는 유일한 도구가 아니라는 점은 주목할 가치가 있습니다.

'LangChain'FAISS 설치 (CPU version -> !pip install faiss-cpu)


In [ ]:
!pip install pypdf
!pip install faiss-cpu #CPU version 설치

In [ ]:
# Load PDF
loader = PyPDFLoader("MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("matlib에 대한 어떤 이야기가 있나요?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])